In [3]:
import numpy as np 
from sklearn.model_selection import RepeatedKFold
import nibabel as nib
from scipy.io import loadmat
from importlib import reload
import pickle
import matplotlib.pyplot as plt 
import seaborn as sns

In [1]:
from sklearn.model_selection import train_test_split
import os, sys
import numpy as np
import pdb
import argparse
import json 
import subprocess
import pandas as pd
import glob
from scipy.stats import ttest_1samp, ttest_ind, zscore
from statsmodels.stats.multitest import fdrcorrection, multipletests
from mne.stats import fdr_correction
from sklearn.preprocessing import OneHotEncoder

## FDR Correction

In [2]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

import mne
from mne import io
from mne.datasets import sample
from mne.stats import bonferroni_correction, fdr_correction

print(__doc__)

Automatically created module for IPython interactive environment


## Prep Atlases

### Combine Harvard-Oxford atlases for cortical and sub-cortical regions

In [32]:
basepath = '/sysapps/ubuntu-applications/fsl/6.0.5.2/fsl/data/atlases/'

In [33]:

subcort = nib.load(basepath + 'HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz')
subcort_data = np.array(subcort.get_fdata(), dtype=int)
subcort_names = pd.read_csv('../../../data/atlases/HO-Subcort-ROIs.csv')

In [34]:
cort = nib.load(basepath + 'HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz')
cort_names = pd.read_csv('../../../data/atlases/HO-Cort-ROIs.csv')

In [35]:
cb = nib.load(basepath + 'Cerebellum/Cerebellum-MNIflirt-maxprob-thr25-1mm.nii.gz')
cb_data = np.array(cb.get_fdata(), dtype=int)
cb_names = pd.read_csv('../../../data/atlases/HO-CB-ROIs.csv')
# cb_data[cb_data>0] = 1 # Binarize, as CB is needed as one region

In [36]:
out_names = pd.DataFrame([],columns=cort_names.columns)
out_names = out_names.append(cort_names, ignore_index=True)

In [37]:
out_data = np.array(cort.get_fdata(), dtype=int)

In [38]:
c1 = np.array([[i,(out_data==i).sum()] for i in np.unique(out_data) if i!=0 ])
print(-np.sort(-c1[:,1]))

[120948  78196  70031  55269  45499  45246  44787  41973  37827  32849
  26880  25157  23450  23252  21879  21449  20844  19542  19228  18720
  16041  15745  14953  14538  13831  12680  11973  11954  11759  11674
  11317   9956   9934   9503   9021   8739   8032   7796   6952   6488
   5871   5698   5385   5313   4901   4730   4482   2081]


In [39]:
c2 = np.array([[i,(subcort_data==i).sum()] for i in np.unique(subcort_data) if i!=0])
print(-np.sort(-c2[:,1]))

[512675 509101 250063 249575  38610  10385  10211   9299   8283   6397
   6397   5688   5575   4127   3949   2848   2476   2133   2118    756
    666]


In [40]:
c3 = np.array([[i,(cb_data==i).sum()] for i in np.unique(cb_data) if i!=0])
print(-np.sort(-c3[:,1]))

[21097 20988 15898 15207 13113 11958  6765  6761  6571  5970  5784  5695
  5677  5670  5630  5598  5221  5170  2975  1779   984   954   926   658
   580   367   113]


In [41]:
print(-np.sort(-c3[:,1]).sum())

178109


In [42]:
np.unique(out_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [43]:
np.unique(subcort_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21])

In [44]:
# Replace label numbers for subcortical data to append to cortical
offset = 48 # There are 0-47 labels in cortical regions
remove_indices = np.array([0,1,11,12])
for li in range(21):
    if li in remove_indices:
        print('Skipping %d'%li)
        continue
    out_data[subcort_data==(li+1)] = li+1+offset
    

Skipping 0
Skipping 1
Skipping 11
Skipping 12


In [45]:
np.unique(out_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69])

In [46]:
# Remove cortical and white matter voxels from subcortical map
remove_values = remove_indices + 1 + offset
# remove_voxels = np.zeros(subcort_data.shape, dtype=bool)
adder = 0
new_map = {}
for v in range(1+offset,70):
    if v in remove_values:
        adder += 1
        out_data[out_data==v] = 0
        print('%d,%s will be removed'%(v, subcort_names['name'][v-offset-1]))
    else:
        print('Replacing %d,%s with %d,%s'%
              (v-adder,subcort_names['name'][v-offset-1-adder], v,subcort_names['name'][v-offset-1]))
        out_data[out_data==v] = v - adder
        names_row = {ci:subcort_names.iloc[v-offset-1][ci] for ci in subcort_names.columns}
        names_row['index'] = v - adder - 1
        out_names = out_names.append(names_row, ignore_index=True)
        new_map[v-1] = v-1 - adder  # -1 because of indices
    # remove_voxels = np.logical_or(remove_voxels, out_data == rv)
# out_data[np.where(remove_voxels==True)] = 0

49,Left Cerebral White Matter will be removed
50,Left Cerebral Cortex  will be removed
Replacing 49,Left Cerebral White Matter with 51,Left Lateral Ventricle
Replacing 50,Left Cerebral Cortex  with 52,Left Thalamus
Replacing 51,Left Lateral Ventricle with 53,Left Caudate
Replacing 52,Left Thalamus with 54,Left Putamen
Replacing 53,Left Caudate with 55,Left Pallidum
Replacing 54,Left Putamen with 56,Brain-Stem
Replacing 55,Left Pallidum with 57,Left Hippocampus
Replacing 56,Brain-Stem with 58,Left Amygdala
Replacing 57,Left Hippocampus with 59,Left Accumbens
60,Right Cerebral White Matter will be removed
61,Right Cerebral Cortex  will be removed
Replacing 58,Left Amygdala with 62,Right Lateral Ventricle
Replacing 59,Left Accumbens with 63,Right Thalamus
Replacing 60,Right Cerebral White Matter with 64,Right Caudate
Replacing 61,Right Cerebral Cortex  with 65,Right Putamen
Replacing 62,Right Lateral Ventricle with 66,Right Pallidum
Replacing 63,Right Thalamus with 67,Right Hippocampus
Re

In [47]:
len(out_names)

65

In [48]:
65 in out_names['index']

False

In [49]:
# Add Cerbellar Regions with new indices for some of the combined regions.
cboffset = len(out_names)
new_cb_map = {}
for i in range(27):
    li = cb_names.iloc[i]['newindex']
    out_data[cb_data == i+1] = li+1 + cboffset
    
    if not li+cboffset in out_names['index']:
        ignore_cols = ['newindex','newname','newshortname']
        names_row = {ci:cb_names.iloc[i][ci] for ci in cb_names.columns if ci not in ignore_cols}
        names_row['index'] = cb_names.iloc[i]['newindex'] + cboffset
        names_row['shortname'] = cb_names.iloc[i]['newshortname']
        names_row['name'] = cb_names.iloc[i]['newname']    
        out_names = out_names.append(names_row, ignore_index=True)
    new_cb_map[i+offset] = i

In [50]:
np.unique(out_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76])

In [51]:
np.unique(out_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76])

In [52]:
outpath = '../../../data/atlases/'

In [53]:
nib.save(nib.Nifti1Image(np.array(out_data, dtype=int), affine=cort.affine, header=cort.header), outpath+'HO-CortSubcortCB-atlas.nii.gz')

In [54]:
out_names.to_csv(outpath+'HO-CortSubcortCB-ROIs.csv', index=False)

In [55]:
with open(outpath + 'new_subcort_roi_map.json','w+') as f:
    json.dump(new_map, f)

In [56]:
with open(outpath + 'new_CB_roi_map.json','w+') as f:
    json.dump(new_cb_map, f)

In [29]:
np.unique(out_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76])

### Generate Mask for each ROI

In [68]:
nx,ny,nz = out_data.shape
nm = np.unique(out_data).max()
out_masks = []

for i in range(nm):
    curmask = np.zeros(out_data.shape, dtype=int)
    curmask[out_data==(i+1)] = i+1
    out_masks.append(curmask)
out_masks = np.array(out_masks,dtype=int)
out_masks = np.moveaxis(out_masks, 0, -1)

In [72]:
nib.save(nib.Nifti1Image(np.array(out_masks, dtype=int), affine=cort.affine, header=cort.header), outpath+'HO-CortSubcortCB-atlas-ROImasks.nii.gz')

## Saliency

In [4]:
import utils as ut 
import summarize as smr
from models import AN3Ddr_lowresMax
import torch
from torch.autograd import Variable
from scipy.ndimage import gaussian_filter

In [ ]:
a = nib.load('/data/users2/ibatta/data/features/lowresSMRI/ADNI/random_subject.nii.gz')

In [ ]:
b = np.zeros(a.get_fdata().shape)

In [ ]:
nib.save(nib.Nifti1Image(b, affine=a.affine, header=a.header), 'check_nibsave.nii')

In [1]:
saltype = 'fsal_raw'
basedir = '../out/results/latest/'
config_string = 'mt_AN3DdrlrMx_fkey_lT1_scorename_xyz_iter_*_nc_2_rep_*_bs_32_lr_0.0001_espat_20' 
configlist_file = '../in/config_keys/allcombos_baseline_clf_nc2'
files_key = basedir + config_string + '/' + saltype + '.pkl'

In [2]:
import summarize as smr 

In [ ]:
smr.summarize_all_saliencies(configlist_file, 'labels_3way',10,-1)

mt_AN3DdrlrMx_fkey_lT1_scorename_age_iter_*_nc_2_rep_*_bs_32_lr_0.0001_espat_20
fsal_raw


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
smr.summarize_all_saliencies(configlist_file, 'labels_3way',10,0)

In [ ]:
outfile = '../out/saliency_stats/' + config_string.replace('*','X') + '/tstats.nii.gz'

In [ ]:
smr.summarize_saliency(config_string, 'labels_3way',outfile, saltype=saltype)

In [ ]:
cfgpath = glob.glob(basedir + config_string)[0] + '/config.pkl' 
cfg = ut.loadCfg(cfgpath)
masks = ut.loadMasks(cfg)

In [ ]:
def load_masked_saliency_maps(config_string, masks, saltype, normalize=True):
    """
    Loads saliency maps from all the files in the mentioned config string into a single list of length nch

    Args:
        config_string (str): keywords to be used as configuration string to search from models outputs directory
        
        
    """

    basedir = '../out/results/latest/'
    
    nch, nx, ny, nz = masks.shape
    mr = masks.reshape([nch,nx*ny*nz])
    
    out_data = [[] for ch in range(nch)]
    for salfile in glob.glob(basedir+config_string+'/'+saltype+'.pkl'):
        with open (salfile, 'rb') as f:
            sal_data = pickle.load(f)
        assert np.all(sal_data.shape[1:] == masks.shape)
        nsub, _, _, _, _ = sal_data.shape
        sal_data = sal_data.reshape([nsub,nch,nx*ny*nz])
        for ch in range(nch):
            out_data[ch].append(sal_data[:,ch,mr[ch]==1])
    for ch in range(nch):
        out_data[ch] = np.vstack(out_data[ch])
        if normalize:
            out_data[ch] = ut.normalize_image(out_data[ch], method='zscore', axis=1)
        
    return out_data        



In [ ]:
md = load_masked_saliency_maps(config_string, masks, saltype, normalize=False)

In [ ]:
mdc = [md[i][0] for i in range(len(md))]

In [ ]:
mdc[0].shape , mdc[1].shape

In [ ]:
mdc = [ut.normalize_image(mdc[i]) for i in range(len(mdc))]

In [ ]:
from scipy.stats import zscore

In [ ]:
b = np.array([[ 0.3148,  0.0478,  0.6243,  0.4608],
              [ 0.7149,  0.0775,  0.6072,  0.9656],
              [ 0.6341,  0.1403,  0.9759,  0.4064],
              [ 0.5918,  0.6948,  0.904 ,  0.3721],
              [ 0.0921,  0.2481,  0.1188,  0.1366]])

In [ ]:
b.shape

In [ ]:
b.mean(axis=1)

In [ ]:
zscore(b, axis=1).mean(axis=1)

In [ ]:
c = np.array([b,b])

In [ ]:
c.shape

In [ ]:
zscore(c, axis=2).mean(axis=2)

In [ ]:

def save_masked_maps(data, masks, outfilepath):
    # mask is a 4-D np array with shape (nch, nx, ny, nz)
    # data is a list (and not np.array!) of vectors, each with different length depending on non-zero voxels of the corresponding mask
    # outfilepath should be a path with .nii or .nii.gz as extension.
    
    nch, nx, ny, nz = masks.shape
    assert len(data) == nch
    mr = masks.reshape(nch, nx*ny*nz)
    
    cur_out = np.zeros([nch, nx*ny*nz])
    for ch in range(nch): 
        cur_out[ch,mr[ch]==1] = data[ch]
    
    cur_out =  np.moveaxis(cur_out.reshape([nch,nx,ny,nz]), 0, -1)
    nib.save(nib.Nifti1Image(cur_out, np.eye(4)), outfilepath)


In [ ]:
save_masked_maps(mdc, masks, '../out/check_save.nii')

In [ ]:
print(files_key)

In [ ]:
maskfile = "/data/users2/ibatta/data/masks/ADNI/ADNI_T1_lowres.nii.gz"
mask = nib.load(maskfile).get_fdata()

In [ ]:
mr = mask.reshape(np.prod(mask.shape))

In [ ]:
salfile = glob.glob(basedir+config_string+'/'+saltype+'.pkl')[0]
with open(salfile, 'rb') as f:
    sal_data = pickle.load(f)
sal_data = ut.normalize_5D(np.abs(sal_data), method='zscore')
n, nch, nx, ny, nz = sal_data.shape
print(sal_data.shape)


In [ ]:
zscore(sal_data[0,0], axis=None).mean()

In [ ]:
sal_data[0,1].mean()

In [ ]:
sd = sal_data.reshape([n, nch, nx*ny*nz])[:,0]

In [ ]:
sd.shape

In [ ]:
sd[:,mr==1].shape

In [ ]:
t, p = ttest_1samp((sd[:,mr==1]), 0, axis=0)

In [ ]:
sal_data.shape

In [ ]:
sal_data[:,0].shape

In [ ]:
t.shape, p.shape

In [ ]:
nch*nx*ny*nz

In [ ]:
p.min(), p.max()

In [ ]:
# out  = multipletests(p.reshape([nch*nx*ny*nz]), method='fdr_bh')
out  = multipletests(p, method='fdr_bh')

In [ ]:
cd = plt.hist(pc,bins=100, density=True, cumulative=False, histtype='stepfilled')

In [ ]:
# (rej, pc) = fdrcorrection(p.reshape([nch*nx*ny*nz]))
(rej, pc) = fdrcorrection(p)


In [ ]:
pc.shape

In [ ]:
a = np.zeros([nx*ny*nz])

In [ ]:
a[mr==1] = pc

In [ ]:
a = a.reshape([1,nx,ny,nz])

In [ ]:

nib.save(nib.Nifti1Image(a, np.eye(4)), 
             'check.nii')

In [ ]:
pc.min(), pc.max()

In [ ]:
np.max(pc)

In [ ]:
sal_data.mean()

In [ ]:
np.max(sal_data), np.min(sal_data)

In [ ]:
(rej, pc) = fdr_correction(p.reshape([nch*nx*ny*nz]))


In [ ]:

rej = rej.reshape([nch,nx,ny,nz])
pc = pc.reshape([nch,nx,ny,nz])

In [ ]:
pc[rej==True][-10:], p[rej==True][-10:]

In [ ]:
(np.isnan(p)).sum()


In [ ]:
(rej==False).sum()

In [ ]:
fp = '/data/users2/ibatta/projects/deepsubspace/out/results/mt_AN3DdrlrMx_fkey_lT1_scorename_labels_3way_iter_10_nc_2_rep_0_bs_32_lr_0.0001_espat_20/fimsal.pkl'
with open(fp,'rb') as f:
    sal = pickle.load(f)


In [ ]:
sal.shape

In [ ]:
def myminmax(X, axis=0):
    return X*np.sum(X)

c = np.apply_over_axes(myminmax, a, [0,1])

In [ ]:
sal = np.moveaxis(sal,0,-1)
nib.save(nib.Nifti1Image(sal[0], np.eye(4)), 'test.nii')

In [ ]:
c[0,0,0,0,0]

In [ ]:
b.shape

In [ ]:
reload(ut)

In [ ]:
mdpath = '../out/modelout/model_state_dict.pt'
cfpath = '../out/modelout/config.pkl'

# model = AN3Ddr_lowresMax(num_classes=2, num_channels=1, num_groups=1)

cfg = ut.loadCfg(cfpath)
cfg.ml = '../out/modelout/'
model = ut.loadNet(cfg)

model.load_state_dict(torch.load(mdpath))
model.eval()


In [ ]:
dataloader = ut.loadData(cfg, 'te')

In [ ]:
for _, data in enumerate(dataloader, 0):
    inputs, labels = data
    break

In [ ]:
inputs.shape

In [ ]:
tempmask = np.ones(inputs.shape[2:])

In [ ]:
tempmask.shape

In [ ]:
inputs = inputs.cpu()
model = model.cpu()
ut.run_saliency('../out/modelout/','BP', inputs, model, tempmask, 'labels_3way','clx')

In [ ]:
a,b,c,d = ut.sensitivity_analysis(model, inputs, tempmask, cuda=False)

In [ ]:
d = '/data/users2/ibatta/data/features/fmrimeasures/ADNI/nii/002_S_6007/Axial_rsfMRI__Eyes_Open_/2017-03-31_10_40_49.0/S551365/rest/'

f1 = nib.load(d+'swarest1_tsavg.nii').get_fdata()
f2 = nib.load(d+'swarest1_ALFF.nii').get_fdata()


In [ ]:
np.prod(f1.shape)

In [ ]:
(f1-f2 > 10).sum()

In [ ]:
f1 = (f1 - f1.min()) / (f1.max()-f1.min())
f2 = (f2 - f2.min()) / (f2.max()-f2.min())


In [ ]:
(f1-f2 > 0.01).sum()

In [ ]:
fl = '/data/qneuromark/Data/ADNI/Updated/fMRI/ADNI/177_S_6335/Axial_MB_rsfMRI__Eyes_Open_/2020-07-22_12_56_01.0/S951207/rest/swarest1.nii'
dl = nib.load(fl).get_fdata()


In [ ]:
basedir='/data/qneuromark/Data/ADNI/Updated/fMRI/Results/GIGICA/041_S_6401/Axial_rsfMRI__Eyes_Open_/2018-06-12_13_51_03.0/S694594/rest/swarest1/'
f1 = 'adni_aa__sub01_component_ica_s1_.nii'
f2 = 'adni_aa__ica_c1-1.mat'



In [ ]:
d1 = nib.load(basedir + f1).get_fdata()

In [ ]:
d2 = loadmat(basedir + f2)

In [ ]:
d2.keys()

In [ ]:
d1.shape

In [ ]:
d2['ic'].shape, d2['tc'].shape

In [ ]:
53*63*52, 66529*194

##  QC 

### Constant Valued Scans

In [ ]:
dataset = 'ADNI'
filemapper = json.load(open('../in/filemapper.json','r'))

measures = list(filemapper['filename']['ADNI'].keys())

nanfiles = []
numnans = []
badfiles = []

for m in measures: 
    basepath_mapper = filemapper['basepathmapper'][dataset][m]
    basedir = filemapper['basedir'][dataset][basepath_mapper]
    # basedir = '/data/qneuromark/Data/ADNI/Updated/fMRI/ADNI/'
    filename = filemapper['filename']['ADNI'][m]
    # filename = 'swarest1.nii'
    for dir in open('../in/%s_MMR180.csv'%basepath_mapper,'r+').read().split('\n')[:-1]:
        fullpath = basedir + dir + filename
        scandata = nib.load(fullpath).get_fdata()
        if np.isnan(scandata).sum() > 0:
            nn = np.isnan(scandata).sum()
            print(nn)
            print(fullpath)
            nanfiles.append(fullpath)
            numnans.append(nn)
        if np.abs(np.std(scandata)) < 0.001:
            print(np.abs(np.std(scandata)))
            print(fullpath)
            badfiles.append(fullpath)




In [ ]:
with open('../in/badfiles_measures.txt','w+') as f:
    f.write('\n'.join(badfiles))

with open('../in/numnans_measures.txt','w+') as f:
    f.write('\n'.join(['%s,%d'%(nanfiles[i],numnans[i]) for i in range(len(nanfiles))]))
    

In [ ]:
len(['ALFF', 'DCw', 'DCb', 'fALFF', 'KccReHo', 'VMHC', 'lT1', 'ALFF,DCw,DCb,fALFF,KccReHo,VMHC,lT1', 'ALFF,lT1', 'DCw,lT1', 'DCb,lT1', 'fALFF,lT1', 'KccReHo,lT1', 'VMHC,lT1'])

### Imputations

In [ ]:
with open('../in/vars.txt', 'r+') as f:
    scores = f.read().split('\n')[:-1]
scores_file = '../in/analysis_SCORE_MMR180d.csv'
df = pd.read_csv(scores_file)


In [ ]:
print(df['age'].dtype)

In [ ]:

for score in scores:
    print('%s, %s'%(score,df[score].dtype))
    dt = df[score].dtype
    if not 'float' in str(dt):
        df[score] = df[score].fillna(df[score].mode())
    else:
        df[score] = df[score].fillna(df[score].median())

df.to_csv('../in/analysis_SCORE_MMR180d_imputed.csv',  index=False)

In [ ]:
output_file = '../out/performances/baseline/allcombos_baseline_reg.png '

In [ ]:
pval_file = '.'.join(output_file.split('.')[:-1]) + '_pvals.' + output_file.split('.')[-1]

In [ ]:
fns = ['tsavg','tsmedian','tsmax','tsmin']
    # fns.remove('hT1')
    # fns.remove('PerAF')
    # fns.append(','.join(fns)) #multimodal with all features into DL model
    fns += [fi+',lT1' for fi in fns if 'lT1' not in fi] # multimodal with 2 modalities: with one fmri measure with low-res smri 
    # fns.append(','.join([','.join(fns) for i in range(8)] )) # Testing 56 groups for future  
    print(fns)

    # # Map iter value (slurm taskID) to training sample size (tss) and crossvalidation repetition (rep)
    fv, rv = np.meshgrid(np.arange(len(fns)), np.arange(cfg.nReps))
    fv = fv.reshape((1, np.prod(fv.shape)))
    rv = rv.reshape((1, np.prod(rv.shape)))
    fkey = fns[fv[0][cfg.iter]]
    rep = rv[0][cfg.iter]
    print(fkey, rep)
    cfg.fkey = fkey
    cfg.nch = len(fkey.split(','))
    cfg.rep = rep
    print(cfg.iter, cfg.tss, cfg.rep)


In [ ]:
fm = json.load(open('../in/filemapper.json','r'))

In [ ]:
fns = list(fm['filename']['ADNI'].keys())

In [ ]:
len(fns)

In [ ]:
fns = ['tsavg','tsmedian','tsmax','tsmin']
# fns.remove('hT1')
# fns.remove('PerAF')
# fns.append(','.join(fns)) #multimodal with all features into DL model
fns += [fi+',lT1' for fi in fns if 'lT1' not in fi] # multimodal with 2 modalities: with one fmri measure with low-res smri 

In [ ]:
i = 8
fv, rv, sv = np.meshgrid(np.arange(len(fns)), np.arange(10), np.arange(8))
fv = fv.reshape((1, np.prod(fv.shape)))
rv = rv.reshape((1, np.prod(rv.shape)))
sv = sv.reshape((1, np.prod(sv.shape)))

for i in range(80):
    fkey = fns[fv[0][i]]
    rep = rv[0][i]
    scr = sv[0][i]
    print(fkey, rep, scr)

In [ ]:
fv.shape, rv.shape, sv.shape

In [ ]:
i = 8
fns.remove('hT1')
fns.remove('PerAF')

fv, rv = np.meshgrid(np.arange(len(fns)), np.arange(10))
fv = fv.reshape((1, np.prod(fv.shape)))
rv = rv.reshape((1, np.prod(rv.shape)))

for i in range(80):
    fkey = fns[fv[0][i]]
    rep = rv[0][i]
    
    print(i,fkey, rep)

In [ ]:
def summarize_config(config_string):
    """
    Computes the set of test accuracy for the string corresponding to the config_string. 
    The config strings should represent one of the directory. 
    """

    basedir = '../out/results/latest/'
    cmd = 'cat %s%s/test.csv | grep -v acc_te | cut -f2 -d\",\"'%(basedir,config_string)
    output_stream = os.popen(cmd)
    test_accs = np.array(output_stream.read().split('\n')[:-1], dtype=float)
    return test_accs

In [ ]:
config_string = 'AN3Ddr_lowresMax_fkey_ALFF_scorename_labels_3way_iter_*_nc_2_rep_*_bs_32_lr_1e-05_espat_20'

In [ ]:
t = summarize_config(config_string)

In [ ]:
len(t)

In [ ]:
t = summarize_config('AN3Ddr_lowresMax_fkey_ALFF_scorename_labels_3way_iter_*_nc_2_rep_*_bs_32_lr_1e-05_espat_20')

In [ ]:

n = 30
k_te = 4
k_va = 5
nreps = 3

all_labels = np.array(list(range(0,30)), dtype=int) // 10

# selelct specific labels if needed for binary classification
select_labels = [0,2]
labels = np.array([li for li in all_labels if li in select_labels], dtype=int)
sli = np.array([i for i in range(len(all_labels)) if all_labels[i] in select_labels], dtype=int)


for ri in np.arange(nreps):
	tri, test_indices = train_test_split(np.arange(len(labels)), test_size=1.0/k_te, shuffle=True, stratify=labels, random_state=108+ri)
	tr_idx, val_idx = train_test_split(np.arange(len(tri)), test_size=1/k_va, shuffle=True, stratify=labels[tri], random_state=108+ri)
	train_indices, val_indices = tri[tr_idx], tri[val_idx]

	print(all_labels[sli[train_indices]], all_labels[sli[val_indices]], all_labels[sli[test_indices]])
    


In [ ]:
np.loadtxt('../out/temp1.txt').mean()

In [ ]:
np.loadtxt('../out/temp2.txt').mean()

In [ ]:


	np.savetxt(outdir+'/tr_r'+str(ri)+'.csv', train_indices, fmt='%d')
	np.savetxt(outdir+'/va_r'+str(ri)+'.csv', val_indices, fmt='%d')
	np.savetxt(outdir+'/te_r'+str(ri)+'.csv', test_indices, fmt='%d')



In [ ]:
from sklearn.model_selection import train_test_split

tr, va = train_test_split(np.arange(n), test_size=0.2)

In [ ]:
tr, va

In [ ]:
import pandas as pd 
# Try Merge
fl = '../in/lowresfilelist_ADNI_SMRI.txt'
sm = '../in/analysis_SCORE_SMRI_fpL.csv'

df_fl = pd.read_csv(fl)
df_sm = pd.read_csv(sm).iloc[idx]


In [ ]:
for iter in range(12):
    tv, rv = np.meshgrid([], np.arange(3))
    tv = tv.reshape((1, np.prod(tv.shape)))
    rv = rv.reshape((1, np.prod(tv.shape)))
    tss = tv[0][iter]
    rep = rv[0][iter]
    print(tss, rep)

In [ ]:
import nibabel as nib 

In [ ]:
df_sm

In [ ]:
df_sm

In [ ]:
idx = np.loadtxt('../in/SampleSplits/ADNI/3way/te_r0.csv', dtype=int)

In [ ]:
idx

In [ ]:
df_sm = df_sm.iloc[idx]

In [ ]:
df_sm